## 1. Create a sample design in Metal

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings

In [3]:
design = designs.DesignPlanar()
gui = MetalGUI(design)

from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.interconnects.meandered import RouteMeander

In [4]:
design.variables['cpw_width'] = '15 um'
design.variables['cpw_gap'] = '9 um'

### In this example, the design consists of 4 qubits and 4 CPWs

In [5]:
# Allow running the same cell here multiple times to overwrite changes
design.overwrite_enabled = True

## Custom options for all the transmons
options = dict(
    # Some options we want to modify from the defaults
    # (see below for defaults)
    pad_width = '425 um', 
    pocket_height = '650um',
    # Adding 4 connectors (see below for defaults)
    connection_pads=dict(
        readout = dict(loc_W=+1,loc_H=-1, pad_width='200um'),
        bus1 = dict(loc_W=-1,loc_H=+1, pad_height='30um'),
        bus2 = dict(loc_W=-1,loc_H=-1, pad_height='50um')
    )
)

## Create 4 transmons

q1 = TransmonPocket(design, 'Q1', options = dict(
    pos_x='+2.42251mm', pos_y='+0.0mm', **options))
q2 = TransmonPocket(design, 'Q2', options = dict(
    pos_x='+0.0mm', pos_y='-0.95mm', orientation = '270', **options))
q3 = TransmonPocket(design, 'Q3', options = dict(
    pos_x='-2.42251mm', pos_y='+0.0mm', orientation = '180', **options))
q4 = TransmonPocket(design, 'Q4', options = dict(
    pos_x='+0.0mm', pos_y='+0.95mm', orientation = '90', **options))

RouteMeander.get_template_options(design)

options = Dict(
        lead=Dict(
            start_straight='0.2mm',
            end_straight='0.2mm'),
        trace_gap='9um',
        trace_width='15um')

def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, asymmetry='0 um', flip=False, fillet='90um'):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        fillet=fillet,
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        total_length=length)
    myoptions.update(options)
    myoptions.meander.asymmetry = asymmetry
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

asym = 140
cpw1 = connect('cpw1', 'Q1', 'bus2', 'Q2', 'bus1', '6.0 mm', f'+{asym}um')
cpw2 = connect('cpw2', 'Q3', 'bus1', 'Q2', 'bus2', '6.1 mm', f'-{asym}um', flip=True)
cpw3 = connect('cpw3', 'Q3', 'bus2', 'Q4', 'bus1', '6.0 mm', f'+{asym}um')
cpw4 = connect('cpw4', 'Q1', 'bus1', 'Q4', 'bus2', '6.1 mm', f'-{asym}um', flip=True)

gui.rebuild()
gui.autoscale()

## 2. Render into Ansys Q3D

In [24]:
fourq_q3d = design.renderers.q3d

Skip the next cell if Ansys is already open and with a project loaded

In [37]:
#fourq_q3d.open_ansys()
fourq_q3d.open_ansys(path='C:\Program Files\AnsysEM\AnsysEM21.1\Win64')

**Wait for Ansys to open up completely before proceeding to the next cell**

In [38]:
fourq_q3d.connect_ansys()
fourq_q3d.add_q3d_design("Q3dMetalDesign")

INFO 12:52PM [connect_project]: Connecting to Ansys Desktop API...
INFO 12:52PM [load_ansys_project]: 	Opened Ansys App
INFO 12:52PM [load_ansys_project]: 	Opened Ansys Desktop v2021.1.0
INFO 12:52PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/LeeJaeho/Documents/Ansoft/
	Project:   Project4
INFO 12:52PM [connect_design]: No active design found (or error getting active design).
INFO 12:52PM [connect]: 	 Design not detected in project. Is there a design in your project? 🤔 

INFO 12:52PM [connect_design]: 	Opened active design
	Design:    Q3dMetalDesign [Solution type: Q3D]
WARNING 12:52PM [connect_setup]: 	No design setup detected.


Render a single qubit with 2 endcaps

In [39]:
fourq_q3d.render_design(['Q1'], [('Q1', 'readout'), ('Q1', 'bus1'), ('Q1', 'bus2')])

## 3. Create a new solution setup in Ansys Q3D and obtain the capacitance matrix

In [40]:
#     Add a solution setup:
fourq_q3d.add_q3d_setup()
#     Example of how to change defaults (just an example. We will not use this setup):
#fourq_q3d.add_q3d_setup(name = 'Setup', max_passes = 20)

Optional keyword parameters and their defaults, listed below, represent parameters in this Q3D solution setup. Running fourq_q3d.add_q3d_setup() is equivalent to going to the Project Manager panel in Ansys, right clicking on Analysis within the active Q3D design, selecting "Add Solution Setup...", and choosing/entering default values in the resulting popup window. To easily keep track of different solution setups, give each of them a different name, otherwise, Ansys will simply append numbers to the end of duplicate names. <br><br>
freq_ghz: float = 5. <br>
name: str = "Setup" <br>
save_fields: bool = False <br>
enabled: bool = True <br>
max_passes: int = 15 <br>
min_passes: int = 2 <br>
min_converged_passes: int = 2 <br>
percent_error: float = 0.5 <br>
percent_refinement: int = 30 <br>
auto_increase_solution_order: bool = True <br>
solution_order: str = 'High' <br>
solver_type: str = 'Iterative <br><br>
These defaults can be modified by including the keywords and their new values inside the parentheses. For example, to increase the maximum number of passes from the default value of 15 up to 20 while leaving all else unchanged, replace the cell above with the following: <br><br>
fourq_q3d.add_q3d_setup(max_passes = 20)

In [41]:
# Analyze said solution setup.
fourq_q3d.analyze_setup("Setup")

INFO 12:53PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 12:53PM [analyze]: Analyzing setup Setup


In [42]:
# Using the analysis results, get its capacitance matrix as a dataframe.
fourq_q3d.get_capacitance_matrix()

,bus1_connector_pad_Q1,bus2_connector_pad_Q1,ground_main_plane,pad_bot_Q1,pad_top_Q1,readout_connector_pad_Q1
bus1_connector_pad_Q1,47.81116,-0.42249,-32.38145,-1.50814,-12.42209,-0.20677
bus2_connector_pad_Q1,-0.42249,51.70372,-34.49845,-12.99176,-1.79524,-1.00014
ground_main_plane,-32.38145,-34.49845,232.85650,-31.03545,-37.08001,-35.67206
pad_bot_Q1,-1.50814,-12.99176,-31.03545,92.85160,-27.59688,-17.51171
pad_top_Q1,-12.42209,-1.79524,-37.08001,-27.59688,83.60249,-2.16620
readout_connector_pad_Q1,-0.20677,-1.00014,-35.67206,-17.51171,-2.16620,57.63952


The optional keyword parameters of get_capacitance_matrix are given below: <br><br>
variation: str = '' <br>
solution_kind: str = 'AdaptivePass' <br>
pass_number: int = 3

In [43]:
# Run lumped oscillator model (LOM) simulations and save results in dict_lom.b
dict_lom = fourq_q3d.lumped_oscillator_vs_passes(12.31, 2, 3, 7.0, [0.1, 0.1, 0.1], 9)
dict_lom

Pass number:  1
[3, 4] [5 0 1]
Predicted Values

Transmon Properties
f_Q 5.751241 [GHz]
EC 353.645796 [MHz]
EJ 13.273404 [GHz]
alpha -417.920940 [MHz]
dispersion 136.533417 [KHz]
Lq 0.000055 [nH]
Cq 54.772963 [fF]
T1 37.150672 [us]

**Coupling Properties**

tCqbus1 6.334904 [fF]
gbus1_in_MHz 108.297183 [MHz]
χ_bus1 -4.772408 [MHz]
1/T1bus1 4284.038990 [Hz]
T1bus1 37.150676 [us]

tCqbus2 -5.052783 [fF]
gbus2_in_MHz -1.240541 [MHz]
χ_bus2 -0.000084 [MHz]
1/T1bus2 0.000186 [Hz]
T1bus2 854670425.883671 [us]

tCqbus3 5.006688 [fF]
gbus3_in_MHz 1.229221 [MHz]
χ_bus3 -0.000082 [MHz]
1/T1bus3 0.000183 [Hz]
T1bus3 870483340.976520 [us]
Bus-Bus Couplings
gbus1_2 0.094309 [MHz]
gbus1_3 0.102818 [MHz]
gbus2_3 0.001084 [MHz]
Pass number:  2
[3, 4] [5 0 1]
Predicted Values

Transmon Properties
f_Q 5.667709 [GHz]
EC 342.689918 [MHz]
EJ 13.273404 [GHz]
alpha -403.561052 [MHz]
dispersion 104.913667 [KHz]
Lq 0.000057 [nH]
Cq 56.524068 [fF]
T1 40.034899 [us]

**Coupling Properties**

tCqbus1 6.731029 [fF

,fQ,EC,EJ,alpha,dispersion,gbus,chi_in_MHz,χr MHz,gr MHz
1,5.751241,353.645796,13.273404,-417.92094,136.533417,"[108.29718318411753, -1.2405406205837783, 1.22...","[-4.772407670889936, -8.395420641943842e-05, -...",4.772408,108.297183
2,5.667709,342.689918,13.273404,-403.561052,104.913667,"[112.20219168528992, -1.3719906530500392, 1.11...","[-4.45910069383321, -0.00010194676265934142, -...",4.459101,112.202192
3,5.576534,330.959668,13.273404,-388.288083,77.976994,"[111.29064396907027, -1.4051930124271585, 1.16...","[-3.7922836957293113, -0.00010610902121792412,...",3.792284,111.290644
4,5.524531,324.374985,13.273404,-379.760434,65.542687,"[111.00290359134131, -1.4055708155383666, 1.18...","[-3.4803792183726925, -0.00010569951784796476,...",3.480379,111.002904
5,5.464644,316.886639,13.273404,-370.102092,53.435731,"[113.08543185172468, -1.4180669859995698, 1.17...","[-3.2984020981918394, -0.00010704756750851794,...",3.298402,113.085432
6,5.427737,312.321897,13.273404,-364.235131,47.008788,"[114.27995126401626, -1.43701362230614, 1.1953...","[-3.1881674181878847, -0.00010958943324190832,...",3.188167,114.279951
7,5.402946,309.277112,13.273404,-360.330359,43.088528,"[115.02700555985716, -1.4526952022409099, 1.20...","[-3.1140846096762624, -0.00011176410145520343,...",3.114085,115.027006
8,5.389238,307.600806,13.273404,-358.183525,41.048534,"[115.47473891685145, -1.4605080416810508, 1.21...","[-3.076011752854409, -0.00011284141023388667, ...",3.076012,115.474739


Using capacitance matrices obtained from each pass, save the many parameters of the Hamiltonian of the system. lumped_oscillator_vs_passes takes in the following parameters: <br><br>
Lj_nH: float <br>
Cj_fF: float <br>
N: int <br>
fr: Union[list, float] <br>
fb: Union[list, float] <br>
maxPass: int <br>
variation: str = '' <br>
solution_kind: str = 'AdaptivePass' <br>
g_scale: float = 1

dist_analysis allows one to calculate dissipation and other properties. It is used for the convergence plots below.

In [47]:
fourq_q3d.plot_convergence_main(dict_lom);
fourq_q3d.plot_convergence_chi(dict_lom)

INFO 01:13PM [hfss_report_full_convergence]: Creating report for variation 0


Design "Q3dMetalDesign" info:
	# eigenmodes    0
	# variations    1


<Figure size 990x385 with 2 Axes>

In [46]:
fourq_q3d.options

{'Lj': '10nH',
 'Cj': 0,
 '_Rj': 0,
 'max_mesh_length_jj': '7um',
 'project_path': None,
 'project_name': None,
 'design_name': None,
 'ansys_file_extension': '.aedt',
 'x_buffer_width_mm': 0.2,
 'y_buffer_width_mm': 0.2}

In [35]:
fourq_q3d.disconnect_ansys()

Warning! 6 COM references still alive
HFSS will likely refuse to shut down


### Code to swap rows and columns in capacitance matrix
from qiskit_metal.analyses.quantization.lumped_capacitive import df_reorder_matrix_basis

df_reorder_matrix_basis(fourq_q3d.get_capacitance_matrix(), 1, 2)